# Practice Module: Convergence & Analysis Tools

<p>It's very difficult oftentimes to get reproducibility in molecular dynamics. So "convergence" is the idea that if you run a simulation for LONG enough, it will ultimately "converge" at one point. So how do you determine if your data has converged? Some ways:
<p>1. Run all simulations in triplicate (which is just good experimental practice) because then you can report standard deviation of all measurements. <p>2. Run your simulation for a long, long time  <p> Unfortunately, monolayer systems are difficult, because they haven't been studied in the computational community for very long. You'll find some papers that ran theirs for 5ns and some than ran for 150 ns. So it's up to us to decide HOW LONG we should be running these things, and this module will help you do *ACTUAL* analysis for me on this question. <p>To do this, we'll look at some NAMD simulations that I've completed.

This practice will look like:
    1. Using MDTraj / basic python functions
    2. Reading and writing files 
    3. Plotting data with matplotlib

In [ ]:
## import needed methods
import numpy as np #for math
from decimal import *
import mdtraj as md ## you'll need this
import pytraj as pt ## also this
import matplotlib.pyplot as plt ##very important for plotting
from matplotlib import cm
%matplotlib inline 
###always comment your code

<p>The files you will need for this analysis are the three replicate files for surface area 20.5, labeled SA20.5, SA20.5-2, and SA20.5-3. They are in this file directory: 

    filedir1 = '/gpfs/amarolab/abbyflabby/MIX/MIX1234_NaCl0.4/'

<p>Copy and paste that line into a new cell (hit the + at the top) and then run the code (shift + Enter). This is setting your first variable which is your file directory. 

<p> NAMD uses a shit ton of files to run ANYTHING, but to load each simulation for analysis, you need two things: a topology file and a trajectory file.
<p> NAMD topology file: ".psf"
<p> NAMD trajectory file: ".dcd"
<p> I'll load SA19 with mdtraj just to show you what I need you to do with EACH simulation. Notice that the file structures I use and the file names you will notice are always the same, for consistency.

In [ ]:
sim1 = md.load(filedir1+"SA19/charmm-gui/namd/step7.1_production.dcd", top=filedir1+"SA19/charmm-gui/step5_assembly.xplor_ext.psf")
print(sim1)
#after printing you should see <mdtraj.Trajectory with 3698 frames, 40451 atoms, 7673 residues, and unitcells>
#3698 frames == 3698*2 ps == 7396 ps => 7.396 ns 

If you print the trajectory, you'll see its specs. Go ahead and load your three replicates below, give them USEFUL and NOT BULKY names. For example: traj1, traj2, traj3

We can talk about input files later, but according to the NAMD 7.1_production.inp input script, a dcd file (FRAME) was printed every 2ps (that's PICO-second). How many ns is this? how many nanoseconds long are your trajectories?

## Using MDTraj

<p>I haven't given you ANY information about these simulations--You should figure out what they're made of. mdtraj has some great online documentation: http://mdtraj.org/latest/examples/introduction.html<p>To get you started, I will first count how many water molecules there are, and see whether there are any ions in solution.<p><i>Number of waters

In [ ]:
### sim1 has already been loaded, so I want to specify water only
#selection language for MDtraj
water1 = sim1.top.select("water")
print(water1)
#notice that water is an array... it should print like this:
#[18772 18773 18774 ..., 40348 40349 40350]
##These numbers are the INDICES of all ATOMS that make up water in the trajectory
#when coding it's always useful to print stuff to see what it looks like and what form it's in

##I want to know HOW MANY water molecules there are... so I want to print the LENGTH of this array. 
print(len(water1))
#21579
##recall that these are the indices of each ATOM.... to find number of molecules, we need to divide by three (3 atoms per molecule)
water_residues1 = len(water1)/3
print(water_residues1)
#7193

<i>Number of ions

In [ ]:
## several ways of doing this. If you know that the ions could only be Na+ Or Cl- it's relatively easy. 
##FYI for THESE simulations the only salts added are NaCl
##select for sodium
sod1 = sim1.top.select("resname SOD")
print(sod1)
print(len(sod1))
#now chloride
cla1 = sim1.top.select("resname CLA")
print(cla1)
print(len(cla1))

<i> Making a new trajectory

In [ ]:
##I want to make a new trajectory with JUST items that I've selected
##if I want a trajectory of just water, I like to do it this way:
## I already defined a water selection from the above code: water1 = sim1.top.select("water1")
##turns out this gives me all the indices I want, which is the input for the next function:
water_traj1 = sim1.atom_slice(water1)
print(water_traj1)
##another way to get # water molecules:
print(water_traj1.n_residues)
##<mdtraj.Trajectory with 3698 frames, 21579 atoms, 7193 residues, and unitcells>

Now you try!<p> 1. How many lipids are there? <p>2. How many of each lipid are there? <p>3.Can you create a trajectory of JUST lipids?

In [ ]:
##YOu will need to know what you're looking for.... so ... 
###here is function to get a list of residue NAMES
###see if you can understand why it works
#defining a function
def getResidues(traj):#this function takes a trajectory as an input!
    residues1 = []
    for i in traj.top.residues:
        check = 'false'
        for j in residues1:
            if i.name == j:
                check = 'true' ##why do I use "=" instead of "=="?
                break
        if len(residues1) == 0:
            residues1.append(i.name)
        elif check == 'false': ##why do I use "==" instead of "="? 
            residues1.append(i.name)
    return residues1 #output
##using the function
getResidues(sim1)

In [ ]:
##How might I get total number of residues? 
palp_selection1 = sim1.top.select('resname PALP') #select 
palp_traj1 = sim1.atom_slice(palp_selection1) #cut new trajectory
print(palp_traj.n_residues) #print number of residues

In [ ]:
#number of each lipid

In [ ]:
#total lipids

In [ ]:
#trajectory containing ALL lipids only 

## Read and write files in python

We don't always have cute analysis modules like MDtraj to analyze our data for us, so we have to go straight to the source--our simulation ouptut files-- to get data. These are usually found in log files. In the same location as my dcd file, I will locate my log file and create a variable:

In [ ]:
logfile1 = filedir1+"SA19/charmm-gui/namd/step7.1_production.log"

Let's navigate to this logfile in terminal and open it to see what it looks like. 

    cd /gpfs/amarolab/abbyflabby/MIX/MIX1234_NaCl0.4/SA19/charmm-gui/namd/
    vim step7.1_production.log
<p> The top of the file should look something like:

    Charm++: standalone mode (not using charmrun)
    Converse/Charm++ Commit ID: v6.6.1-rc1-1-gba7c3c3-namd-charm-6.6.1-build-2014-Dec-08-28969
    Warning> Randomization of stack pointer is turned on in kernel, thread migration may not work! Run 'echo 0 > /proc/sys/kernel/randomize_va_space' as root to disable it, or try run with '+isomalloc_sync'.
    CharmLB> Load balancer assumes all CPUs are same.

<p>We need to practice using python to "read" this file, line by line, so we can eventually get it to extract the data we want. To do this, copy and paste the below lines into a new cell, where "r" designates we want to read the file: 

    log1 = open(logfile, "r")

In [ ]:
log1 = open(logfile1, "r")

<p>So this creates a variable that is actually formatted like an array of lines in the file. So we can iterate over the array with a for loop: 

    for line in log1:
         print(line)

<p>The above code will print out EVERY SINGLE LINE in the file. You can run it if you want to, but it will take a while.. There are lots of lines. 
<p>So what do we want to do with this new skill? Well. We want to extract data that gets printed. In the log file, NAMD outputs the following in this format:

    ETITLE:      TS           BOND          ANGLE          DIHED          IMPRP
              ELECT            VDW       BOUNDARY           MISC        KINETIC
              TOTAL           TEMP         TOTAL2         TOTAL3        TEMPAVG
           PRESSURE      GPRESSURE         VOLUME       PRESSAVG      GPRESSAVG
           
    ENERGY:    2375      1625.1660      9744.4764      5148.3221       113.6684        -107154.1206       -75.2390         0.0000         0.0000     25827.5733         -64770.1534       296.1745    -90597.7267    -64580.2604       296.5764            137.4891       134.0194   1126319.9984        50.2003        50.1848

    PRESSURE: 2500 29.0588 81.0596 62.4582 81.0593 19.6978 -67.639 62.4589 -67.6394 102.935
    GPRESSURE: 2500 6.95917 74.8063 33.0297 50.6297 24.2815 -49.3801 51.3257 -21.8493 108.988
    PRESSAVG: 2500 33.9646 -9.53635 8.45501 -9.53633 8.48039 -3.26265 8.45503 -3.26266 39.3728
    GPRESSAVG: 2500 33.8341 -9.66428 8.75072 -9.05383 8.66055 -3.436 8.38506 -2.83566 39.2882
    TIMING: 2500  CPU: 18.1282, 0.00707493/step  Wall: 18.2818, 0.00711786/step, 2.56986 hours remaining, 329.734375 MB of memory in use.                                                         

Let's say I want to measure the pressure, GPRESSAVG, over the course of the entire simulation. I will want the timestep (TS) as well as the value for GPRESSAVG. My strategy is to create an array of x(time) values and an array of y(pressure) values, for simplicity. So, in pseudocode:

    #initialize empty x, y arrays
    #read each line in the file
    #if the line starts with "GPRESSAVG," we want to pay attention to this line
    #otherwise, we don't care about the line
    #Break the energy line into its individual numerical components using a python method...
    #We want to extract the timestep (index 1) and the GPRESSAVG (index 2) into their respective arrays, then move to the next line!

In [ ]:
title = 'GPRESSAVG'
#initialize empty x, y arrays
gpress_x1 = []
gpress_y1 = []
#read each line in the file
with open (logfile1, 'r') as log1:
    for line in log1:
        if line.startswith(title):
            ##the first thing that gets printed here is 'GPRESSAVG'
            ##the second thing is the timestep
            ##the third thing is one of the values for the pressure tensor. We can just say this is the value I want.
            #to append the timestep to the x array & the tensor value to the y array:
            gpress_x1.append(float(line.split(" ")[1]))##1 index is timestep. 
            gpress_y1.append(float(line.split(" ")[2]))##2 index is the first tensor value
            ##google what split does!
print(gpress_x1)

Now it's your turn to write a function that grabs some values from the log file(s) from your SA20.5 trajectories. You will be grabbing "TOTAL3" energies from the ENERGY list (shown above) (search the NAMD docs to figure out what TOTAL3 includes and what its units are!) as the y axis and timesteps as the x-axis. 
<p> As you can see from the NAMD output reproduced below, TOTAL3 is the 15th item in the line that starts with ETITLE, corresponding to the 15th item in the line that starts with ENERGY. Can you figure out how to write a function that extracts the 15th item in the line, as well as the timestep, TS, into two new arrays?

    ETITLE:      TS           BOND          ANGLE          DIHED          IMPRP
              ELECT            VDW       BOUNDARY           MISC        KINETIC
              TOTAL           TEMP         TOTAL2         TOTAL3        TEMPAVG
           PRESSURE      GPRESSURE         VOLUME       PRESSAVG      GPRESSAVG
    
    ENERGY:    1000         0.0000         0.0000         0.0000         0.0000
        -97022.1848      9595.3175         0.0000         0.0000     14319.5268
        -73107.3405       300.2464    -73076.6148    -73084.1411       297.7598
          -626.5205      -636.6638    240716.1374      -616.5673      -616.6619

In [ ]:
##code here!
##there MIGHT be multiple log files, start with the first one 
#and then you can write scripts to add the next log file to the array 

## Using MATPLOTLIB for making graphs

To analyze our new data that we've stored in arrays, it's useful to be able to plot it! I'll show you how to generate a scatter plot with your data.
<p><i>Scatter Plot of GPRESSAVG with TIME

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.scatter(gpress_x1, gpress_y1)

In the plot above, you can see that there are quite a lot of points, and in the x-axis, the numbers are so long that they run into each other. How can we fix that? Let's figure out what the largest number is in the x axis. This is the LAST item in the array OR we can use numpy (np) to find the max value of the array.

    #NUMPY
    max_np = np.max(gpress_x1)
    print max_np
    
    #last item in array:
    max_arr = gpress_x1[-1]
    print max_arr
    
    ##result is 3698500.0

In [ ]:
##your code here

We can do a couple of things to fix this, but I suggest breaking it down into smaller numbers. Two ways. 
<p> One:The largest number is approximately 3.6 x 10^6. If we divide all numbers by 1,000,000 or 100,000 then each number will be smaller and the xaxis labels won't run into each other. We can then specify units in the x-axis title.
<p>Two: change to nanoseconds by dividing by 500,000. We'll do this option.

In [ ]:
new_x1 = np.array(gpress_x1)/500000
##changed into a numpy array so we can do math on it like that
print(new_x1)
##Now the largest value is ~7.397

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.scatter(new_x1, gpress_y1, marker='.', s=2)
#I've added some extra stuff here like marker type and size to give you a hint on how to do this yourself
plt.xlabel('time / ns')
plt.ylabel('pressure / bar')
plt.title('Average Pressure (measured per 2ps)')

<i> Scatter plot of ENERGY vs time

Now you get to try your hand at coding this. Here is your task:
<p> 1. Plot your energy data the same way I've plotted this. #energy is given in kcal/mol

<p>2. Make a NEW plot, except on this plot you will have THREE sets of data on the SAME plot, each with proper labels:

    A. ENERGY (measured per 2 picoseconds) #remember that a data point is grabbed EVERY 2 PICOSECONDS
    B. Energy averaged over every 0.01 ns, including error (standard deviation) bars
    C. Energy averaged over every 0.1 ns, including error (standard deviation) bars